In [5]:
from dash import Dash, html, dcc, callback, Output, Input, State
from dash import dash_table
from dash_bootstrap_templates import load_figure_template
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd

In [6]:
athlete_events = pd.read_csv("./Data/athlete_events.csv")
athlete_events.head(2)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN


In [19]:
medals = athlete_events[athlete_events['Medal'].isin(['Gold', 'Silver', 'Bronze'])]
medals_count = medals.groupby(['Sport', 'Team', 'Medal']).size().reset_index(name='Count')

all_sports = medals['Sport'].unique()

app = Dash(__name__)

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1("Olympic Games Dashboard", className='text-center text-primary mx-3')
            ], width=12, style={'text-align': 'center'})
    ]),
      
    dbc.Row([
        dbc.Col([
            dcc.Dropdown(
                id='sport_selector',
                options=[{'label': sport, 'value': sport} for sport in all_sports],
                value=all_sports[0]
            ),
            dcc.Graph(id='medal_graph')
        ])
    ]),
    
    dbc.Row([
        dbc.Col([
            dcc.Dropdown(
                id='sport_selector2',
                options=[{'label': sport, 'value': sport} for sport in all_sports],
                value=all_sports[0]
            ),
            dcc.Graph(id='age_graph')
        ])
    ])
])

@callback(
    Output('medal_graph', 'figure'),
    Input('sport_selector', 'value')
)
def update_medal_graph(selected_sport):
    medals_df = medals_count[medals_count['Sport'] == selected_sport]
    fig = px.bar(medals_df, 
                 x="Team", y="Count", 
                 color="Medal", 
                 title=f"Medals for {selected_sport}", 
                 template="simple_white", 
                 labels={"Team": "Country", "Count": "Number of medals"})
    fig.update_xaxes(tickangle=-45)
    return fig

@callback(
    Output('age_graph', 'figure'),
    Input('sport_selector2', 'value')
)
def update_age_graph(selected_sport):
    sport_df = athlete_events[athlete_events['Sport'] == selected_sport]
    age_distribution = sport_df.groupby('Age').size().reset_index(name='Count')
    fig = px.bar(age_distribution, 
                 x="Age", y="Count", 
                 color="Age", 
                 title=f"Age distribution for {selected_sport}", 
                 template="simple_white",
                 labels={"Count": "Number of athletes"})
    return fig

app.run(debug=True, jupyter_mode='external')

Dash app running on http://127.0.0.1:8050/
